In [272]:
import json
import os
import openai
import pandas as pd
from tqdm.notebook import tqdm
from openai_interface import GPT_Turbo
import tiktoken
pd.set_option('display.max_rows', 400)

In [262]:
data_path = './data/impact_theory_data.json'
with open(data_path) as f:
    data = json.load(f)

In [273]:
openai.api_key = os.environ['OPENAI_API_KEY']
encode = tiktoken.encoding_for_model('gpt-3.5-turbo-1106')

In [257]:
gpt = GPT_Turbo()

In [ ]:
system_msg = 'You are an expert at creating summaries of podcast shows'

prompt = "Given a podcast episode transcript surrounded by triple backticks, and the guest of the show, write a summary of the episode between 500 and 750 tokens in length:\n\
Guest: {guest}\n\
```{transcript}```\
"

system_msg_count = len(encode.encode(system_msg))
prompt_count = len(encode.encode(prompt))
total_input = system_msg_count + prompt_count
total_input

data[34]

In [418]:
def single_sum(num: int, prompt: str=prompt):
    d = data[num]
    content = ' '.join(d['content'].split()[:6000])
    guest = d['guest']
    summary_prompt = prompt.format(guest=guest, transcript=content)
    try:
        response = gpt.get_completion_from_messages(prompt=summary_prompt, temperature=0.15, system_message=system_msg)
        return response
    except Exception as e:
        return e

In [419]:
def update(num: int, response: str):
    data[num]['summary'] = response

In [430]:
for x in range(108,385):
    guest = data[num]['guest']
    resp = single_sum(num)
    print(f'GUEST: {guest}')
    print()
    print(f'SUMMARY: {resp}')
    update(num, resp)
    time.sleep(15)

GUEST: Ed Mylett

SUMMARY: In this episode, Ed Mylett shares his personal journey and insights on identity, growth, and success. He discusses how he overcame challenges from his childhood, including being bullied and dealing with a chaotic home life. Mylett emphasizes the importance of identifying and leveraging one's natural gifts to shift one's identity and achieve success. He also talks about the significance of receiving feedback and criticism as a catalyst for personal growth.

Mylett shares his experience of transitioning from a shy, bullied kid to a successful athlete and entrepreneur. He emphasizes the role of mentors and coaches in helping individuals recognize their natural talents and linking those gifts to their success. He also discusses the importance of having compelling emotional reasons to drive one's standards and goals, citing personal experiences with health and fitness as motivating factors.

The conversation delves into Mylett's journey as a speaker and the evolut

KeyboardInterrupt: 

In [427]:
update(num, resp)

In [428]:
data[num]['summary'], data[num]['guest']

("In this episode, Ed Mylett shares his personal journey of overcoming challenges and finding success. He discusses how he shifted his identity from being a shy, bullied kid to a successful entrepreneur by identifying and using his natural gifts. Mylett emphasizes the importance of linking one's giftedness to success and how it can change one's identity.\n\nHe also talks about the significance of receiving feedback and criticism, and how it has driven him to constantly strive for personal growth. Mylett believes that one's past should never be bigger than their future and encourages listeners to be hungry for improvement.\n\nMylett shares his experience of overcoming fear and anxiety in public speaking, emphasizing the importance of being passionate and believing in what one is saying. He also stresses the power of touching dreams and making them familiar to the mind, as well as the manipulation of time to increase productivity and value.\n\nThe episode concludes with Mylett's advice o

In [386]:
# summaries = []
# token_count = 0
# rate_limit = 160000
# batch = 1
# for d in tqdm(data):
#     content = ' '.join(d['content'].split()[:5000])
#     input_count = len(encode.encode(content))
#     token_count += (input_count + total_input)
#     print(f'Total Token Count: {token_count}')
#     if token_count > rate_limit * batch:
#         time.sleep(60)
#         batch += 1
#     guest = d['guest']
#     summary_prompt = prompt.format(guest=guest, transcript=content)
#     try:
#         response = gpt.get_completion_from_messages(prompt=summary_prompt, system_message=system_msg)
#         summaries.append(response)
#     except Exception as e:
#         summaries.append(e)

In [387]:
# start = 97
# n = 20

# print(f'Handling range {start} - {start+n}')
# summaries = []
# token_count = 0
# total_input = system_msg_count + prompt_count
# for d in tqdm(data[start:start+n]):
#     content = ' '.join(d['content'].split()[:5000])
#     input_count = len(encode.encode(content))
#     token_count += (input_count + total_input)
#     print(f'Total Token Count: {token_count}')
#     guest = d['guest']
#     summary_prompt = prompt.format(guest=guest, transcript=content)
#     try:
#         response = gpt.get_completion_from_messages(prompt=summary_prompt, system_message=system_msg)
#         summaries.append(response)
#     except Exception as e:
#         summaries.append(e)

In [432]:
for i, sum in enumerate(data):
    print()
    print(f'Guest: {data[i]["guest"]}')
    print(f'Episode #: {i}')
    print()
    print(f'Summary: {data[i]["summary"]}')
    print('-'*150)


Guest: Ian Bremmer
Episode #: 0

Summary: In this episode, Ian Bremmer discusses the rise of big tech as a third superpower and the potential dangers and opportunities it presents. He highlights the immense power held by tech companies in shaping society, the economy, and national security, emphasizing their sovereignty over the digital world. Bremmer expresses concerns about the growing influence of AI and its potential to outstrip government regulation, leading to a reality where tech companies wield significant power over individuals. He also delves into the risks associated with AI proliferation, including the potential for non-governments to control and misuse the technology, exacerbating social inequalities and disinformation. Bremmer emphasizes the need to address negative externalities and regulate AI to mitigate its adverse impacts. Additionally, he discusses the implications of AI on job displacement and social discontent, particularly for marginalized communities. The conve

KeyError: 'summary'

In [377]:
for i, num in enumerate(range(89,97)):
    print(i, num)
    data[num]['summary'] = summaries[i]

0 89
1 90
2 91
3 92
4 93
5 94
6 95
7 96


In [433]:
with open('first107.json', 'w') as f:
    json.dump(data, f)

In [385]:
ls

1_Data_Preprocessing_Week1_COLAB.ipynb     index_templates.py
1_Data_Preprocessing_Week1_PRACTICE.ipynb  models/
2-Weaviate_Week1.ipynb                     openai_interface.py
3-Reranking_and_Evaluation.ipynb           pca.py
4-Retrieval_Evaluation.ipynb               preprocessing.py
5-Search_Pipeline.ipynb                    prompt_templates.py
README.md                                  ragas.ipynb
__pycache__/                               requirements.txt
app.py                                     reranker.py
assets/                                    retrieval_evaluation.py
class_templates.py                         retriever_pipeline.py
css_templates.py                           stream_demo.py
data/                                      summaries.py
fine_tuning.ipynb                          summarize.ipynb
first80.json                               tiktoken_functions.py
first89.json                               utilities/
first96.json                               weaviate_intro